<td>   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a></td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/integrations/yolo/import_yolov8_annotations.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/integrations/yolo/import_yolov8_annotations.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Import YOLOv8 Annotations
This notebook provides examples of setting up an Annotate Project using annotations generated by the [Ultralytics](https://docs.ultralytics.com/) library of YOLOv8. In this guide, we will show you how to:

1. Import image data rows for labeling

2. Set up an ontology that matches the YOLOv8 annotations

3. Import data rows and attach the ontology to a project

4. Process images using Ultralytics

5. Import the annotations generated

## Set Up

In [ ]:
%pip install -q --upgrade "labelbox[data]"
%pip install -q --upgrade ultralytics

In [ ]:
import labelbox as lb
import labelbox.types as lb_types

import ultralytics
from PIL import Image

import uuid
import io

## API Key and Client
Replace the value of `API_KEY` with a valid [API key](https://docs.labelbox.com/reference/create-api-key) to connect to the Labelbox client.

In [ ]:
API_KEY = None
client = lb.Client(api_key=API_KEY)

## Set Up a YOLOv8 model
Initialize our model for image data rows using `yolov8n-seg.pt`, which supports segmentation masks.

In [ ]:
model = ultralytics.YOLO("yolov8n-seg.pt")

## Example: Import YOLOv8 Annotations

The first few steps of this guide will demonstrate a basic workflow of creating data rows and setting up a project. For a quick, complete overview of this process, see [Quick start](https://docs.labelbox.com/reference/quick-start).

### Import an Image Data Row
In this example, we use YOLOv8 to annotate this [image](https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg), which contains many objects that YOLOv8 can detect. Later in this guide, we will provide more details on the specific annotations.

In [ ]:
global_key = str(uuid.uuid4())

# create data row
data_row = {
    "row_data":
        "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg",
    "global_key":
        global_key,
    "media_type":
        "IMAGE",
}

# create dataset and import data row
dataset = client.create_dataset(name="YOLOv8 Demo Dataset")
task = dataset.create_data_rows([data_row])
task.wait_till_done()

print(f"Errors: {task.errors}")

### Set Up an Ontology and Project

You need to create an ontology and project that match the data rows you are labeling. The ontology needs to include the annotations you want to derive from YOLOv8. Each feature name must be unique because Labelbox does not support ontologies with duplicate feature names at the first level.

We will include bounding boxes, segment masks, and polygon tools to demonstrate converting each type of annotation from YOLOv8. We will also explain class mapping later in this guide.


#### Create an Ontology

In [ ]:
ontology_builder = lb.OntologyBuilder(tools=[
    lb.Tool(tool=lb.Tool.Type.BBOX, name="Vehicle_bbox"),
    lb.Tool(tool=lb.Tool.Type.BBOX, name="Person_bbox"),
    lb.Tool(tool=lb.Tool.Type.RASTER_SEGMENTATION, name="Vehicle_mask"),
    lb.Tool(tool=lb.Tool.Type.RASTER_SEGMENTATION, name="Person_mask"),
    lb.Tool(tool=lb.Tool.Type.POLYGON, name="Vehicle_polygon"),
    lb.Tool(tool=lb.Tool.Type.POLYGON, name="Person_polygon"),
])

ontology = client.create_ontology(
    name="YOLOv8 Demo Ontology",
    normalized=ontology_builder.asdict(),
    media_type=lb.MediaType.Image,
)

#### Create and Set Up a Project

In [ ]:
project = client.create_project(name="YOLOv8 Demo Project",
                                media_type=lb.MediaType.Image)

project.create_batch(name="batch 1", global_keys=[global_key])

project.setup_editor(ontology)

### Export Data Rows and Get Predictions

Now we can export the data row from our project. Then add the row_data and global_key to a list to make our predictions.

#### Export data

In [ ]:
export_task = project.export()
export_task.wait_till_done()

# prediction list we will be populating
url_list = []
global_keys = []


# callback that is ran on each data row
def export_callback(output: lb.BufferedJsonConverterOutput):

    data_row = output.json

    url_list.append(data_row["data_row"]["row_data"])

    global_keys.append(data_row["data_row"]["global_key"])


# check if export has errors
if export_task.has_errors():
    export_task.get_buffered_stream(stream_type=lb.StreamType.ERRORS).start()

if export_task.has_result():
    export_task.get_buffered_stream().start(stream_handler=export_callback)

### Import YOLOv8 Annotations to a Project

Now that you have finished your initial setup, we can create predictions using YOLOv8 and import the annotations into our project. In this step, we will:

1. Define our import functions

2. Create our labels

3. Import our labels as either ground truths or MAL labels (pre-labels)

#### Define Import Functions

YOLOv8 supports a wide range of annotations. In this guide, we only import bounding boxes, polygons, and segment masks that match the ontology we created earlier. The following functions handle each annotation type by navigating through the YOLOv8 result payload and converting it to the Labelbox annotation format.

All these functions support class mapping, which aligns YOLOv8 annotation names with Labelbox feature names. This mapping allows for different names in Labelbox and YOLOv8 and enables common YOLOv8 names to correspond to the same Labelbox feature in our ontology. We will define this mapping first. In our example, we map `bus` and `truck` to the Labelbox feature name `Vehicle` and person to `Person`. We will create a mapping for each tool type.

In [ ]:
bbox_class_mapping = {
    "person": "Person_bbox",
    "bus": "Vehicle_bbox",
    "truck": "Vehicle_bbox",
}
mask_class_mapping = {
    "person": "Person_mask",
    "bus": "Vehicle_mask",
    "truck": "Vehicle_mask",
}
polygon_class_mapping = {
    "person": "Person_polygon",
    "bus": "Vehicle_polygon",
    "truck": "Vehicle_polygon",
}

##### Bounding Box

In [ ]:
def get_yolo_bbox_annotation_predictions(
        yolo_results, model,
        ontology_mapping: dict[str:str]) -> list[lb_types.ObjectAnnotation]:
    """Convert YOLOV8 model bbox prediction results to Labelbox annotations format.

    Args:
        yolo_results (Results): YOLOv8 prediction results.
        model (Model): YOLOv8 model.
        ontology_mapping (dict[<yolo_class_name>: <labelbox_feature_name>]): Allows mapping between YOLOv8 class names and different Labelbox feature names.
    Returns:
        list[lb_types.ObjectAnnotation]
    """
    annotations = []

    for yolo_result in yolo_results:
        for bbox in yolo_result.boxes:
            class_name = model.names[int(bbox.cls)]

            # ignore bboxes that are not included in our mapping
            if not class_name in ontology_mapping.keys():
                continue

            # get bbox coordinates
            start_x, start_y, end_x, end_y = bbox.xyxy.tolist()[0]

            bbox_source = lb_types.ObjectAnnotation(
                name=ontology_mapping[class_name],
                value=lb_types.Rectangle(
                    start=lb_types.Point(x=start_x, y=start_y),
                    end=lb_types.Point(x=end_x, y=end_y),
                ),
            )

            annotations.append(bbox_source)

    return annotations

##### Segment Mask

In [ ]:
def get_yolo_segment_annotation_predictions(
        yolo_results, model,
        ontology_mapping: dict[str:str]) -> list[lb_types.Label]:
    """Convert YOLOV8 segment mask prediction results to Labelbox annotations format

    Args:
        yolo_results (Results): YOLOv8 prediction results.
        model (Model): YOLOv8 model.
        ontology_mapping (dict[<yolo_class_name>: <labelbox_feature_name>]): Allows mapping between YOLOv8 class names and different Labelbox feature names.
    Returns:
        list[lb_types.ObjectAnnotation]
    """
    annotations = []

    for yolo_result in yolo_results:
        for i, mask in enumerate(yolo_result.masks.data):
            class_name = model.names[int(yolo_result.boxes[i].cls)]

            # ignore segment masks that are not included in our mapping
            if not class_name in ontology_mapping.keys():
                continue

            # get binary numpy array to byte array. You must resize mask to match image.
            mask = (mask.numpy() * 255).astype("uint8")
            img = Image.fromarray(mask, "L")
            img = img.resize(
                (yolo_result.orig_shape[1], yolo_result.orig_shape[0]))
            img_byte_arr = io.BytesIO()
            img.save(img_byte_arr, format="PNG")
            encoded_image_bytes = img_byte_arr.getvalue()

            mask_data = lb_types.MaskData(im_bytes=encoded_image_bytes)
            mask_annotation = lb_types.ObjectAnnotation(
                name=ontology_mapping[class_name],
                value=lb_types.Mask(mask=mask_data, color=(255, 255, 255)),
            )
            annotations.append(mask_annotation)

    return annotations

##### Polygon

In [ ]:
def get_yolo_polygon_annotation_predictions(
        yolo_results, model, ontology_mapping: dict[str:str]) -> list[lb.Label]:
    """Convert YOLOv8 model results to Labelbox polygon annotations format.

    Args:
        yolo_result (Results): YOLOv8 prediction results.
        model (Model): YOLOv8 model.
        ontology_mapping (dict[<yolo_class_name>: <labelbox_feature_name>]): Allows mapping between YOLOv8 class names and different Labelbox feature names.
    Returns:
        list[lb_types.ObjectAnnotation]
    """
    annotations = []
    for yolo_result in yolo_results:
        for i, coordinates in enumerate(yolo_result.masks.xy):
            class_name = model.names[int(yolo_result.boxes[i].cls)]

            # ignore polygons that are not included in our mapping
            if not class_name in ontology_mapping.keys():
                continue

            polygon_annotation = lb_types.ObjectAnnotation(
                name=ontology_mapping[class_name],
                value=lb_types.Polygon(points=[
                    lb_types.Point(x=coordinate[0], y=coordinate[1])
                    for coordinate in coordinates
                ]),
            )
            annotations.append(polygon_annotation)

    return annotations

#### Creating our Labels
Now that we have defined our functions to create our Labelbox annotations, we can run each image through YOLOv8 to obtain our predictions and then use those results with our global keys to create our labels. 

In [ ]:
# label list that will be populated
labels = []

for i, global_key in enumerate(global_keys):
    annotations = []

    # make YOLOv8 predictions
    result = model.predict(url_list[i])

    # run result through each function and adding them to our annotation list
    annotations += get_yolo_bbox_annotation_predictions(result, model,
                                                        bbox_class_mapping)
    annotations += get_yolo_polygon_annotation_predictions(
        result, model, polygon_class_mapping)
    annotations += get_yolo_segment_annotation_predictions(
        result, model, mask_class_mapping)

    labels.append(
        lb_types.Label(data={"global_key": global_key},
                       annotations=annotations))

#### Import Annotations to Labelbox
We have created our labels and can import them to our project. For more information on importing annotations, see [import image annotations](https://docs.labelbox.com/reference/import-image-annotations).

##### Option A: Upload as [Pre-labels (Model Assisted Labeling)](https://docs.labelbox.com/docs/model-assisted-labeling)

This option is helpful for speeding up the initial labeling process and reducing the manual labeling workload for high-volume datasets.

In [ ]:
upload_job = lb.MALPredictionImport.create_from_objects(
    client=client,
    project_id=project.uid,
    name="mal_job" + str(uuid.uuid4()),
    predictions=labels,
)

print(f"Errors: {upload_job.errors}")
print(f"Status of uploads: {upload_job.statuses}")

#### Option B: Upload to a Labeling Project as [Ground Truths](https://docs.labelbox.com/docs/import-ground-truth)

This option is helpful for loading high-confidence labels from another platform or previous projects that just need review rather than manual labeling effort.

In [ ]:
upload_job = lb.LabelImport.create_from_objects(
    client=client,
    project_id=project.uid,
    name="label_import_job" + str(uuid.uuid4()),
    labels=labels,
)

print(f"Errors: {upload_job.errors}")
print(f"Status of uploads: {upload_job.statuses}")

## Clean Up
Uncomment and run the cell below to optionally delete Labelbox objects created.

In [ ]:
# batch.delete()
# project.delete()
# dataset.delete()